In [1]:
!pip install -q transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00


In [2]:
import requests
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Step 1: Set Up Hugging Face API for Response Generation
# We use Falcon-7B via the Hugging Face Inference API.

In [4]:
HF_API_KEY = "Enter_Your_API_Key_From_Hugging_Face"
API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"
HEADERS = {"Authorization": f"Bearer {HF_API_KEY}"}

# Step 2: Set up FAISS for Storing Embeddings and Initialize the Embedding Model
# We use SentenceTransformer to convert text into numerical embeddings.
# FAISS is used to perform efficient similarity search over stored embeddings.

In [16]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
dimension = 384  # MiniLM embedding size
index = faiss.IndexFlatL2(dimension)
document_store = {}

# Step 3: Store Documents
# This function stores text along with its embedding into FAISS and a dictionary.

In [ ]:
def embed_text(text):
    return embedding_model.encode(text).tolist()

def store_document(doc_id, text):
    embedding = np.array([embed_text(text)], dtype=np.float32)
    index.add(embedding)
    document_store[index.ntotal - 1] = text  # Ensure FAISS index matches document_store

# 4: Retrieve Relevant Context
# Given a query, we find the most relevant stored documents.

In [ ]:
def retrieve_context(query, top_k=3):
    query_embedding = np.array([embed_text(query)], dtype=np.float32)
    distances, indices = index.search(query_embedding, top_k)

    retrieved_texts = []
    for idx in indices[0]:
        if idx >= 0 and idx in document_store:  # Avoid negative indices
            retrieved_texts.append(document_store[idx])
        else:
            retrieved_texts.append("")

    return retrieved_texts

# Step 5: Generate Response
# Using retrieved context, we construct a prompt and send it to Falcon-7B.

In [13]:
def generate_response(query):
    context = retrieve_context(query)
    prompt = f"""
    Context: {context}

    Question: {query}

    Provide a relevant and concise answer.
    """
    payload = {"inputs": prompt, "parameters": {"max_new_tokens": 150}}
    response = requests.post(API_URL, headers=HEADERS, json=payload)

    try:
        return response.json()[0]["generated_text"]
    except Exception as e:
        return f"Error in generation: {str(e)}"

# Step 6: Example Usage
# We store sample documents and test the retrieval and generation pipeline.

In [15]:
store_document("1", "The Falcon-7B is an open-source large language model for natural language understanding.")
response = generate_response("What is Falcon-7B?")
print(response)


    Context: ['Falcon-7B is an open-source large language model developed for AI research.', 'The Falcon-7B is an open-source large language model for natural language understanding.', 'Hugging Face provides free API access to Falcon-7B for text generation tasks.']
    
    Question: What is Falcon-7B?
    
    Provide a relevant and concise answer.
    
Falcon-7B is an open-source large language model developed for AI research. It is a deep learning model that can be used for natural language understanding tasks, such as text generation and translation.


In [14]:
store_document("1", "Falcon-7B is an open-source large language model developed for AI research.")
store_document("2", "Hugging Face provides free API access to Falcon-7B for text generation tasks.")

response = generate_response("What is Falcon-7B?")
print(response)



    Context: ['Falcon-7B is an open-source large language model developed for AI research.', 'Hugging Face provides free API access to Falcon-7B for text generation tasks.', '']
    
    Question: What is Falcon-7B?
    
    Provide a relevant and concise answer.
    
Falcon-7B is an open-source large language model developed for AI research. It is a deep learning model that can generate text from a given prompt, and is capable of understanding complex language and generating natural language responses. It is available for free API access from Hugging Face.
